In [93]:
%matplotlib inline

import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [94]:
uri = 'bolt://localhost:7687'
user = 'neo4j'
pwd = '1234'

graph = Graph(uri, auth=('neo4j', pwd))

In [95]:
graph.run('call n10s.graphconfig.init( {  handleMultival: "ARRAY" });')

param,value
handleVocabUris,SHORTEN
handleMultival,ARRAY
handleRDFTypes,LABELS


In [96]:
graph.run('CALL n10s.rdf.import.fetch("file:///D:\RMBI_MMKG_data\FB15K\FB15K_EntityTriples_Final.nt","N-Triples");')

terminationStatus,triplesLoaded,triplesParsed,namespaces,extraInfo,callParams
OK,592213,592213,{ns0: 'http://rdf.freebase.com/ns/'},,{}


In [207]:
cursor = graph.run('MATCH (a)-[*..1]-(b) WHERE a.uri = "http://rdf.freebase.com/ns/m.0c6qh" RETURN b.uri as uri;')
cursor

uri
http://rdf.freebase.com/ns/m.041y2
http://rdf.freebase.com/ns/m.01d_h8
http://rdf.freebase.com/ns/m.0d1pc


In [208]:
result = pd.DataFrame.from_records(cursor, columns=cursor.keys())

In [209]:
result_mid = result['uri'].str.replace('http://rdf.freebase.com/ns/m.','/m/')
result_mid = result_mid[:10]
result_mid

C:\Users\Alex\AppData\Local\Temp/ipykernel_10104/3406298138.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  result_mid = result['uri'].str.replace('http://rdf.freebase.com/ns/m.','/m/')


0      /m/041y2
1     /m/01d_h8
2      /m/0d1pc
3    /m/02hrh1q
4     /m/05zppz
5     /m/0b1f49
6     /m/044lyq
7     /m/02lj6p
8     /m/04q5zw
9     /m/0509bl
Name: uri, dtype: object

In [210]:
import json
import urllib

api_key = open('.api_key').read()
result_name = []
service_url = 'https://kgsearch.googleapis.com/v1/entities:search'

In [211]:
for index, value in result_mid.items():
    params = {
        'ids': value,
        'limit': 10,
        'indent': True,
        'key': api_key,
        }
    url = service_url + '?' + urllib.parse.urlencode(params)
    response = json.loads(urllib.request.urlopen(url).read())
    for element in response['itemListElement']:
        result_name.append(element['result']['name'])

In [212]:
result_name

['Journalism',
 'Film producer',
 'Model',
 'Actor',
 'Male',
 'Brad Grey',
 'Jared Harris',
 'Bernie Mac',
 'Michael De Luca',
 'Julia Ormond']